# Estimate Census data confidence intervals
See [this website](https://registry.opendata.aws/census-2020-amc-mdf-replicates/?utm_campaign=20241024cnmps1&utm_medium=email&utm_source=govdelivery) for more info.

In [ ]:
import duckdb
import os
import pandas as pd
import scipy.stats
import numpy as np

## Download the replicates data 
This runs in the terminal. Check the linked website above for specific download methods. Replace "your-path" in the code.

`s3 cp --no-sign-request --recursive s3://uscb-2020-product-releases/decennial/amc/2020/mdf/2020-dhc-mdf-replicates/ppmf_gzip/ "your_path\PPMF\gz_csv"`

## Create a table for the baseline scenario (de-identified individual responses that make up reported 2020 Census data)

In [ ]:
import duckdb
import os

data_path = "your_path"

# Specify the CSV file path
csv_path = os.path.join(data_path, 'gz_csvs')

# Connect to DuckDB
con = duckdb.connect()

# Step: Drop the temporary table if it exists
drop_query = f"DROP TABLE IF EXISTS baseline_table"

# Execute the query to drop the table
con.execute(drop_query)

# Query to count rows for unique combinations of the four columns
initial_query = f"""
CREATE TABLE baseline_table AS
SELECT 
    CONCAT(TABBLKST, TABBLKCOU, TABTRACTCE, TABBLK) AS GEOID20,
    COUNT(*) AS PPMF0_COUNT 
FROM read_csv_auto('{os.path.join(csv_path,"PPMF20_0_PER.csv.gz")}')
GROUP BY TABBLKST, TABBLKCOU, TABTRACTCE, TABBLK
"""

# Execute the query
con.execute(initial_query)
print("Created baseline table using PPMF0")

## Iterate through each supplied iteration file and calculat the difference from baseline

In [ ]:
# Step 2: Define a function to process each iteration file, calculate COUNT, and calculate the differences
def process_iteration_file(iteration_file, iteration_number):
    print(iteration_file)
    
    # Step: Drop the temporary table if it exists
    drop_query = f"DROP TABLE IF EXISTS iteration_{iteration_number}"

    # Execute the query to drop the table
    con.execute(drop_query)
    
    # Step 2a: Read the iteration file and calculate COUNT, then join it with the baseline_table
    query = f"""
    CREATE TEMPORARY TABLE iteration_{iteration_number} AS
    SELECT 
        CONCAT(b.TABBLKST, b.TABBLKCOU, b.TABTRACTCE, b.TABBLK) AS GEOID20,
        COALESCE(a.PPMF0_COUNT, 0) AS PPMF0_COUNT,
        COUNT(*) AS COUNT,
        {iteration_number} as ITERATION,
        (COUNT(*) - COALESCE(a.PPMF0_COUNT, 0)) AS DIFF,
        ABS(COUNT(*) - COALESCE(a.PPMF0_COUNT, 0)) AS ABS_DIFF,
        POW((COUNT(*) - COALESCE(a.PPMF0_COUNT, 0)), 2) AS SQRD_ERROR
    FROM read_csv_auto('{iteration_file}') b
    LEFT JOIN baseline_table a
    ON a.GEOID20 = CONCAT(b.TABBLKST, b.TABBLKCOU, b.TABTRACTCE, b.TABBLK) 
    GROUP BY b.TABBLKST, b.TABBLKCOU, b.TABTRACTCE, b.TABBLK, a.PPMF0_COUNT, ITERATION
    """

    # Execute the join and calculation query
    con.execute(query)

    # Step 2b: Insert the calculated results into the final result table
    insert_query = f"""
    INSERT INTO result_table (GEOID20, PPMF0_COUNT, ITERATION, COUNT, DIFF, ABS_DIFF, SQRD_ERROR)
    SELECT GEOID20, PPMF0_COUNT, ITERATION, COUNT, DIFF, ABS_DIFF, SQRD_ERROR FROM iteration_{iteration_number}
    """
    
    con.execute(insert_query)

    # Step: Drop the temporary table if it exists
    drop_query = f"DROP TABLE IF EXISTS iteration_{iteration_number}"

    # Execute the query to drop the table
    con.execute(drop_query)

In [ ]:
# Step 3: Create a final result table to store all results
con.execute("DROP TABLE IF EXISTS result_table")

con.execute("""
CREATE TABLE result_table (
    GEOID20 VARCHAR, 
    PPMF0_COUNT BIGINT, 
    ITERATION INT, 
    COUNT BIGINT, 
    DIFF BIGINT, 
    ABS_DIFF BIGINT, 
    SQRD_ERROR DOUBLE
)
""")

In [ ]:
# Step 4: Loop through the iteration files
gz_csvs_folder = os.path.join(data_path, "gz_csvs")

iteration_files = [os.path.join(gz_csvs_folder, f"PPMF20_{i}_PER.csv.gz") for i in range(1,51)]

# Process each iteration file and calculate differences
for i, iteration_file in enumerate(iteration_files, start=1):
    process_iteration_file(iteration_file, i)

# Step 5: Retrieve the final result (optional)
# con.execute("SELECT count(*) FROM result_table").fetchall()
# final_result = con.execute("SELECT * FROM result_table").fetchdf()
# print(final_result)

## Calculate stats on all iterations

In [ ]:
# Step: Group by GEOID20 and calculate the statistics (MEAN, MSE, STD)
query = """
CREATE TABLE summary_stats AS
SELECT 
    GEOID20,
    PPMF0_COUNT,
    AVG(COUNT) AS MEAN,
    AVG(SQRD_ERROR) AS MSE,
    STDDEV(COUNT) AS STD
FROM result_table  
GROUP BY GEOID20, PPMF0_COUNT;
"""

# Execute the query to create the summary statistics table
con.execute(query)

In [ ]:
# Parameters
ci_level = 0.9
degrees_freedom = 5

# Step 1: Set the t-value
t_value = scipy.stats.t.ppf(q=1-(1-ci_level)/2, df=degrees_freedom) 
t_value

In [ ]:
# Step 2: Add intermediate calculations
con.execute("DROP TABLE IF EXISTS intermediate")

intermediate_query = f"""
CREATE TEMPORARY TABLE intermediate AS
SELECT 
    *,
    POWER(MSE, 0.5) AS RMSE,  
    (MEAN - PPMF0_COUNT) AS BIAS,  
    CASE 
        WHEN PPMF0_COUNT > 5 
             AND ABS(MEAN - PPMF0_COUNT) / NULLIF(STD, 0) >= 0.5 
             AND ((MEAN - PPMF0_COUNT) < 0 OR PPMF0_COUNT >= 25)
        THEN TRUE 
        ELSE FALSE 
    END AS MET_CRITERIA, 
    CASE 
        WHEN PPMF0_COUNT > 5 
             AND ABS(MEAN - PPMF0_COUNT) / NULLIF(STD, 0) >= 0.5 
             AND ((MEAN - PPMF0_COUNT) < 0 OR PPMF0_COUNT >= 25)
        THEN PPMF0_COUNT - (MEAN - PPMF0_COUNT) 
        ELSE PPMF0_COUNT 
    END AS POINT_EST
FROM summary_stats;
"""
con.execute(intermediate_query)

## Calculate confidence intervals

In [ ]:
# Step 3: Calculate confidence intervals and create the final table
con.execute("DROP TABLE IF EXISTS result_with_ci")

final_query = f"""
CREATE TABLE result_with_ci AS
SELECT 
    *,
    GREATEST(FLOOR(POINT_EST - {t_value} * RMSE), 0) AS CI_LOW, 
    CEIL(POINT_EST + {t_value} * RMSE) AS CI_HIGH
FROM intermediate;
"""
con.execute(final_query)

In [ ]:
final_results_file = os.path.join(data_path, "parquet/final_results.parquet")

# Step 2: Write the table to a Parquet file
con.execute(f"""
    COPY result_with_ci TO '{final_results_file}' (FORMAT PARQUET)
""")

In [ ]:
result_file = os.path.join(data_path, "parquet/result_table.parquet")

# Step 2: Write the table to a Parquet file
con.execute(f"""
    COPY result_table TO '{result_file}' (FORMAT PARQUET)
""")

baseline_file = os.path.join(data_path, "parquet/baseline_table.parquet")

# Step 2: Write the table to a Parquet file
con.execute(f"""
    COPY baseline_table TO '{baseline_file}' (FORMAT PARQUET)
""")


In [ ]:
summary_stats_file = os.path.join(data_path, "parquet/summary_stats_table.parquet")

# Step 2: Write the table to a Parquet file
con.execute(f"""
    COPY summary_stats TO '{summary_stats_file}' (FORMAT PARQUET)
""")

## Repeat process with housing units

In [ ]:
# With Housing Units Now
# Specify the CSV file path
csv_path = os.path.join(data_path, 'gz_csvs')

# Connect to DuckDB
con = duckdb.connect()

# Step: Drop the temporary table if it exists
drop_query = f"DROP TABLE IF EXISTS baseline_table_hu"

# Execute the query to drop the table
con.execute(drop_query)

In [ ]:
# Query to count rows for unique combinations of the four columns
initial_query = f"""
CREATE TABLE baseline_table_hu AS
SELECT 
    CONCAT(TABBLKST, TABBLKCOU, TABTRACTCE, TABBLK) AS GEOID20,
    COUNT(*) AS PPMF0_COUNT 
FROM read_csv_auto('{os.path.join(csv_path,"PPMF20_0_UNIT.csv.gz")}')
WHERE VACS = 0
GROUP BY TABBLKST, TABBLKCOU, TABTRACTCE, TABBLK
"""

# Execute the query
con.execute(initial_query)
print("Created baseline housing table using PPMF0")

In [ ]:
# Step 2: Define a function to process each iteration file, calculate COUNT, and calculate the differences
def process_iteration_file_hu(iteration_file, iteration_number):
    print(iteration_file)
    
    # Step: Drop the temporary table if it exists
    drop_query = f"DROP TABLE IF EXISTS iteration_{iteration_number}_hu"

    # Execute the query to drop the table
    con.execute(drop_query)
    
    # Step 2a: Read the iteration file and calculate COUNT, then join it with the baseline_table
    query = f"""
    CREATE TEMPORARY TABLE iteration_{iteration_number}_hu AS
    SELECT 
        CONCAT(b.TABBLKST, b.TABBLKCOU, b.TABTRACTCE, b.TABBLK) AS GEOID20,
        COALESCE(a.PPMF0_COUNT, 0) AS PPMF0_COUNT,
        COUNT(*) AS COUNT,
        {iteration_number} as ITERATION,
        (COUNT(*) - COALESCE(a.PPMF0_COUNT, 0)) AS DIFF,
        ABS(COUNT(*) - COALESCE(a.PPMF0_COUNT, 0)) AS ABS_DIFF,
        POW((COUNT(*) - COALESCE(a.PPMF0_COUNT, 0)), 2) AS SQRD_ERROR
    FROM (
        SELECT *
        FROM read_csv_auto('{iteration_file}')
        WHERE VACS = 0
    ) b
    LEFT JOIN baseline_table_hu a
    ON a.GEOID20 = CONCAT(b.TABBLKST, b.TABBLKCOU, b.TABTRACTCE, b.TABBLK) 
    GROUP BY b.TABBLKST, b.TABBLKCOU, b.TABTRACTCE, b.TABBLK, a.PPMF0_COUNT, ITERATION
    """

    # Execute the join and calculation query
    con.execute(query)

    # Step 2b: Insert the calculated results into the final result table
    insert_query = f"""
    INSERT INTO result_table_hu (GEOID20, PPMF0_COUNT, ITERATION, COUNT, DIFF, ABS_DIFF, SQRD_ERROR)
    SELECT GEOID20, PPMF0_COUNT, ITERATION, COUNT, DIFF, ABS_DIFF, SQRD_ERROR FROM iteration_{iteration_number}_hu
    """
    
    con.execute(insert_query)

    # Step: Drop the temporary table if it exists
    drop_query = f"DROP TABLE IF EXISTS iteration_{iteration_number}_hu"

    # Execute the query to drop the table
    con.execute(drop_query)

# Step 3: Create a final result table to store all results
con.execute("DROP TABLE IF EXISTS result_table_hu")

con.execute("""
CREATE TABLE result_table_hu (
    GEOID20 VARCHAR, 
    PPMF0_COUNT BIGINT, 
    ITERATION INT, 
    COUNT BIGINT, 
    DIFF BIGINT, 
    ABS_DIFF BIGINT, 
    SQRD_ERROR DOUBLE
)
""")


In [ ]:
# Step 4: Loop through the iteration files
gz_csvs_folder = os.path.join(data_path, "gz_csvs")

iteration_files = [os.path.join(gz_csvs_folder, f"PPMF20_{i}_UNIT.csv.gz") for i in range(1,51)]

# Process each iteration file and calculate differences
for i, iteration_file in enumerate(iteration_files, start=1):
    process_iteration_file_hu(iteration_file, i)

In [ ]:
# Step: Group by GEOID20 and calculate the statistics (MEAN, MSE, STD)
query = f"""
CREATE TABLE summary_stats_hu AS
SELECT 
    GEOID20,
    PPMF0_COUNT,
    AVG(COUNT) AS MEAN,
    AVG(SQRD_ERROR) AS MSE,
    STDDEV(COUNT) AS STD
-- FROM result_table_hu  
FROM read_parquet('{data_path}/parquet/result_table_hu.parquet')
GROUP BY GEOID20, PPMF0_COUNT;
"""

# Execute the query to create the summary statistics table
con.execute(query)

In [ ]:
# Parameters
ci_level = 0.9
degrees_freedom = 5

# Step 1: Set the t-value
t_value = scipy.stats.t.ppf(q=1-(1-ci_level)/2, df=degrees_freedom) 
t_value

In [ ]:
# Step 2: Add intermediate calculations
con.execute("DROP TABLE IF EXISTS intermediate_hu")

intermediate_query = f"""
CREATE TEMPORARY TABLE intermediate_hu AS
SELECT 
    *,
    POWER(MSE, 0.5) AS RMSE,  
    (MEAN - PPMF0_COUNT) AS BIAS,  
    CASE 
        WHEN PPMF0_COUNT > 5 
             AND ABS(MEAN - PPMF0_COUNT) / NULLIF(STD, 0) >= 0.5 
             AND ((MEAN - PPMF0_COUNT) < 0 OR PPMF0_COUNT >= 25)
        THEN TRUE 
        ELSE FALSE 
    END AS MET_CRITERIA, 
    CASE 
        WHEN PPMF0_COUNT > 5 
             AND ABS(MEAN - PPMF0_COUNT) / NULLIF(STD, 0) >= 0.5 
             AND ((MEAN - PPMF0_COUNT) < 0 OR PPMF0_COUNT >= 25)
        THEN PPMF0_COUNT - (MEAN - PPMF0_COUNT) 
        ELSE PPMF0_COUNT 
    END AS POINT_EST
-- FROM summary_stats_hu;
FROM read_parquet('{data_path}/parquet/summary_stats_table_hu.parquet')
"""
con.execute(intermediate_query)

In [ ]:
# Step 3: Calculate confidence intervals and create the final table
con.execute("DROP TABLE IF EXISTS result_with_ci_hu")

final_query = f"""
CREATE TABLE result_with_ci_hu AS
SELECT 
    *,
    GREATEST(FLOOR(POINT_EST - {t_value} * RMSE), 0) AS CI_LOW_HU, 
    CEIL(POINT_EST + {t_value} * RMSE) AS CI_HIGH_HU
FROM intermediate_hu;
"""
con.execute(final_query)

In [ ]:
result_file_hu = os.path.join(data_path, "parquet/result_table_hu.parquet")

# Write the results_table_hu table to a Parquet file
con.execute(f"""
    COPY result_table_hu TO '{result_file_hu}' (FORMAT PARQUET)
""")


In [ ]:

baseline_file_hu = os.path.join(data_path, "parquet/baseline_table_hu.parquet"

# Write the baseline_table_hu table to a Parquet file
con.execute(f"""
    COPY baseline_table_hu TO '{baseline_file_hu}' (FORMAT PARQUET)
""")


In [ ]:

summary_stats_file_hu = os.path.join(data_path, "parquet/summary_stats_table_hu.parquet")

# Write the summary_stats_hus table to a Parquet file
con.execute(f"""
    COPY summary_stats_hu TO '{summary_stats_file_hu}' (FORMAT PARQUET)
""")

In [ ]:
final_results_file_hu = os.path.join(data_path, "parquet/final_results_hu.parquet")

# Step 2: Write the table to a Parquet file
con.execute(f"""
    COPY result_with_ci_hu TO '{final_results_file_hu}' (FORMAT PARQUET)
""")